In [ ]:
!pip install bing-image-downloader

In [53]:
from bing_image_downloader import downloader
import os
import cv2
import numpy as np
import warnings
from sklearn.metrics import classification_report
warnings.filterwarnings('ignore')
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm

In [57]:
# 폴더명 및 이미지 수 설정
categories = ["green banana", "yellow banana", "black banana"]
image_limit = 20
output_dir = "dataset"

# 모든 이미지를 "banana_images" 폴더에 다운로드
for category in categories:
    downloader.download(category, limit=image_limit, output_dir=output_dir)

[%] Downloading Images to c:\Users\iit80\OneDrive\바탕 화면\강의파일\AI4FW021\dataset\green banana


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://ojaexpress.com/wp-content/uploads/2020/04/WhatsApp-Image-2020-04-09-at-3.45.07-PM.jpeg
[%] File Downloaded !

[%] Downloading Image #2 from https://www.foodincanada.com/wp-content/uploads/2017/10/Green-Bananas-ThinkStock.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://www.beshideshi.com/wp-content/uploads/2019/06/Green-banana.jpg
[!] Issue getting: https://www.beshideshi.com/wp-content/uploads/2019/06/Green-banana.jpg
[!] Error:: Remote end closed connection without response
[%] Downloading Image #3 from http://img.aws.livestrongcdn.com/ls-1200x630/cpi.studiod.com/www_livestrong_com/photos.demandstudios.com/getty/article/103/51/457512575_XS.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://www.treehugger.com/thmb/v_H9Ig8a_FgZJ86g-h2DS9xMuDs=/2121x1414/filters:fill(auto,

In [58]:
def averagecolor(image):
    return np.mean(image, axis=(0, 1))

In [60]:
# 분류를 위해 특성(평균 색상) 및 해당 레이블(빨간색/녹색/검정색/없음) 저장
trainX = []
trainY = []

# 이미지 하위 디렉토리 3개 폴더에 있는 훈련 이미지를 반복합니다.
path = "dataset/"
for label in ('black banana','yellow banana','green banana'):
    print ("Loading training images for the label: "+label)
    
    # 하위 폴더의 모든 이미지를 읽어옵니다.
    for filename in os.listdir(path+label+"/"): 
        img = cv2.imread(path+label+"/"+filename)
        img_features = averagecolor(img)
        trainX.append(img_features)
        trainY.append(label)

Loading training images for the label: black banana
Loading training images for the label: yellow banana
Loading training images for the label: green banana


In [61]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=42)

In [62]:
# KNN 모델 생성
knn_model = KNeighborsClassifier(n_neighbors=3)  # 이웃의 수를 지정합니다. 원하는 값으로 조정하세요.

# 훈련 데이터를 사용하여 모델 훈련
knn_model.fit(X_train, y_train)

# 테스트 데이터를 사용하여 예측 수행
predictedY_knn = knn_model.predict(X_test)
realtestY = y_test

def evaluateaccuracy(filenames,predictedY):
    predictedY = np.array(predictedY)
    if (np.sum(realtestY!=predictedY)>0):
        print ("Wrong Predictions: (filename, labelled, predicted) ")
        # dstack으로 3차원을 만드는데 필요없는 차원이 하나 있어서 삭제 == squeeze
        print (np.dstack([filenames,realtestY,predictedY]).squeeze()[(realtestY!=predictedY)])
    # 전체 예측의 백분율로 일치하는 (정확한) 예측을 계산합니다.
    return "Correct :"+ str(np.sum(realtestY==predictedY)) + ". Wrong: "+str(np.sum(realtestY!=predictedY)) + ". Correctly Classified: " + str(np.sum(realtestY==predictedY)*100/len(predictedY))+"%"

print(predictedY_knn)
# 정확도 평가 (예: 분류 보고서 출력)
print(classification_report(y_test, predictedY_knn))


['black banana' 'black banana' 'green banana' 'green banana'
 'black banana' 'green banana' 'yellow banana' 'green banana'
 'black banana' 'green banana' 'green banana' 'green banana']
               precision    recall  f1-score   support

 black banana       1.00      1.00      1.00         4
 green banana       0.86      1.00      0.92         6
yellow banana       1.00      0.50      0.67         2

     accuracy                           0.92        12
    macro avg       0.95      0.83      0.86        12
 weighted avg       0.93      0.92      0.91        12

